In [35]:
%pip install numpy
%pip install sympy
%pip install scipy
%load_ext autoreload
%autoreload 2
from simplex_two_phases import simplex as simplex_two_phases, _simplex_find_feasible_initial_basis
from simplex_tableau import simplex as simplex_tableau
#from simplex_tableau import markdown_repr_T
from simplex import simplex
from revised_simplex_tableau import revised_simplex_tableau, markdown_repr_T
from utils import array_to_markdown

    
import numpy as np
from pprint import pprint
from IPython.display import Markdown, display
from itertools import combinations
from sympy import symbols, Matrix, solve
from scipy.optimize import linprog


solution_types = {
    -1: 'Unfeasible',
    1: 'Optimal finite solution found',
    2: 'Multiple optimal solutions found',
    3: 'Unbounded'
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Exercicio 4.1

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [36]:
A = np.array([
    [ 1, 1, -1, 0, 0],
    [-2, 3,  0, 1, 0],
    [ 0, 1,  0, 0, 1]
])

B = np.array([
    1,
    6,
    2
])

C = np.array([1, -3, 0, 0, 0])

I = [2, 3, 4]

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Restrições de negatividade violadas

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [37]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [5, 3, 4]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 0.000 | 1.000 | 0.000 | 6.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{5}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{3}\\) | 0.000 | 1.000 | 0.000 | 6.000 | -2.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 1.,  1., -1.])


\\(J = {[0, 1, 2]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{5}\\)
#### Row Operations:

1. \\(R_{X_{5}} \leftarrow \frac{R_{X_{5}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{5}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} + 2.000R_{X_{5}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 0.000R_{X_{5}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & -1.000 & 0.000 & 0.000 & 1.000 \\
-2.000 & 3.000 & 0.000 & 1.000 & 0.000 & 0.000 \\
0.000 & 1.000 & 0.000 & 0.000 & 1.000 & 0.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
1.000\\
6.000\\
2.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['1.000', '0.000', '0.000', '8.000', '2.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([0, 3, 4]\\)

Final non-basic variables set J: \\([5, 1, 2]\\)

\\(\hat{C}_{J}\\): \\([-1.0, 0.0, 0.0]\\)

Number of Iterations: 2



Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Portanto, vamos resolvê-lo à partir da base inicial facítvel obtida na resolução do artificial $ I = [0, 3, 4] $

In [38]:
I = [0, 3, 4]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I, cycle_proof = True)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

[1]




## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|
| Z | 1.000 | 0.000 | 0.000 | 1.000 | 4.000 |
| \\(X_{0}\\) | 1.000 | 0.000 | 0.000 | 1.000 | 1.000 |
| \\(X_{3}\\) | 2.000 | 1.000 | 0.000 | 8.000 | 5.000 |
| \\(X_{4}\\) | 0.000 | 0.000 | 1.000 | 2.000 | 1.000 |


\\(Ĉ_J = \ \\)array([ 4., -1.])


\\(J = {[1, 2]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{0}\\)
#### Row Operations:

1. \\(R_{X_{0}} \leftarrow \frac{R_{X_{0}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 4.000R_{X_{0}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} - 5.000R_{X_{0}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.000R_{X_{0}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | -3.000 | 0.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | -3.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{4}\\) | -1.000 | 0.000 | 1.000 | 1.000 |


## Iteration 1

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | -3.000 | 0.000 | 0.000 | -3.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 1.000 |
| \\(X_{3}\\) | -3.000 | 1.000 | 0.000 | 3.000 |
| \\(X_{4}\\) | -1.000 | 0.000 | 1.000 | 1.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|
| Z | -3.000 | 0.000 | 0.000 | -3.000 | 3.000 |
| \\(X_{1}\\) | 1.000 | 0.000 | 0.000 | 1.000 | -1.000 |
| \\(X_{3}\\) | -3.000 | 1.000 | 0.000 | 3.000 | 3.000 |
| \\(X_{4}\\) | -1.000 | 0.000 | 1.000 | 1.000 | 1.000 |


\\(Ĉ_J = \ \\)array([-4.,  3.])


\\(J = {[0, 2]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{3}\\)
#### Row Operations:

1. \\(R_{X_{3}} \leftarrow \frac{R_{X_{3}}}{3.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 3.000R_{X_{3}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 1.000R_{X_{3}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.000R_{X_{3}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -1.000 | 0.000 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | 0.333 | 0.000 | 1.000 |
| \\(X_{4}\\) | 0.000 | -0.333 | 1.000 | 0.000 |


## Iteration 2

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -1.000 | 0.000 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | 0.333 | 0.000 | 1.000 |
| \\(X_{4}\\) | 0.000 | -0.333 | 1.000 | 0.000 |


Extended Tableau

|   |  | \\(A_I^{-1}\\) |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|
| Z | 0.000 | -1.000 | 0.000 | -6.000 | 1.000 |
| \\(X_{1}\\) | 0.000 | 0.333 | 0.000 | 2.000 | -0.667 |
| \\(X_{2}\\) | -1.000 | 0.333 | 0.000 | 1.000 | -1.667 |
| \\(X_{4}\\) | 0.000 | -0.333 | 1.000 | 0.000 | 0.667 |


\\(Ĉ_J = \ \\)array([ 1., -1.])


\\(J = {[0, 3]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{4}\\)
#### Row Operations:

1. \\(R_{X_{4}} \leftarrow \frac{R_{X_{4}}}{0.667}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{4}}\\)
2. \\(R_{X_{1}} \leftarrow R_{X_{1}} + 0.667R_{X_{4}}\\)
3. \\(R_{X_{2}} \leftarrow R_{X_{2}} + 1.667R_{X_{4}}\\)


Computed Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -0.500 | -1.500 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | -0.500 | 2.500 | 1.000 |
| \\(X_{0}\\) | 0.000 | -0.500 | 1.500 | 0.000 |


## Iteration 3

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | -0.500 | -1.500 | -6.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 1.000 | 2.000 |
| \\(X_{2}\\) | -1.000 | -0.500 | 2.500 | 1.000 |
| \\(X_{0}\\) | 0.000 | -0.500 | 1.500 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & 1.000 & -1.000 & 0.000 & 0.000 \\
-2.000 & 3.000 & 0.000 & 1.000 & 0.000 \\
0.000 & 1.000 & 0.000 & 0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
1.000\\
6.000\\
2.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
1.000 & -3.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['0.000', '2.000', '1.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = -6.000\\)

Optimal Basis: \\([1, 2, 0]\\)

Final non-basic variables set J: \\([4, 3]\\)

\\(\hat{C}_{J}\\): \\([-1.5, -0.5]\\)

Number of Iterations: 4



## Result Comparison through scipy

In [6]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

Result comparison, we found the optimal solution through our tableaus


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -6.0
              x: [ 0.000e+00  2.000e+00  1.000e+00 -0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  2.000e+00  1.000e+00 -0.000e+00
                              0.000e+00]
                 marginals: [ 1.000e+00  0.000e+00  0.000e+00  0.000e+00
                              3.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -0.000e+00 -3.000e+00]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [2, 3, 4]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star.tolist()}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.2

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [ 2,  1, 3, -1,  0, 0],
    [-1,  1, 0,  0, -1, 0],
    [-1, -5, 1,  0,  0, 1]
])

B = np.array([
    3,
    1,
    4
])

C = np.array([1, 3, -1, 0, 0, 0])

I = [3, 4, 5]

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [6, 7, 5]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Portanto, vamos resolvê-lo à partir da base inicial facítvel obtida na resolução do artificial $ I = [2, 1, 5] $ já que nessa base não existem variáveis artificiais

In [ ]:
I = [2, 1, 5]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
# I = [3, 4, 5]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star.tolist()}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.3

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [3,  1, -1, 1,  0,  0],
    [4, -1,  2, 0, -1,  0],
    [2,  3, -2, 0,  0, -1]
])

B = np.array([
    8,
    2,
    4
])

C = np.array([-2, 1, -1, 0, 0, 0])

I = [3, 4, 5]

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [3, 6, 7]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Portanto, vamos resolvê-lo à partir da base inicial facítvel obtida na resolução do artificial $ I = [3, 0, 1] $ já que nessa base não existem variáveis artificiais

In [ ]:
I = [3, 0, 1]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [3, 4, 5]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star.tolist()}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.4

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [1,  2, 1, 1,  0,  0, 0],
    [1, -1, 0, 0, -1,  0, 0],
    [1, -1, 0, 0,  0, -1, 0],
    [1,  3, 1, 0,  0,  0, 1]
])

B = np.array([
    10,
    10,
    6,
    14
])

C = np.array([-4, -5, 3, 0, 0, 0, 0])

I = list(range(3,7))

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [3, 7, 8, 6]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $, o que caracteriza o problema original como factível.

Vemos ainda que uma variável artificial está na base — $ x_7 $ — o que implica a necessidade de sua troca (ou remoção, se impossível, a partir da retirada de sua restrição equivalente) para resolver o problema original.

Vamos avaliar o conjunto $J$ das variáveis não básicas originais onde $ J = [1, 2, 3, 4] $ e seu respectivo $ \hat{C_{J}} = [-3, -1, -1, -1] $

Apesar de todo $\hat{C_{J}}$ ser negativo, podemos inserir uma variável não básica original que piore a solução do problema desde que o conjunto base $I$ resultante mantenha as restrições de factibilidade adequadas.

Pelo critério de entrada, do indice mais à esquerda (critério de bland) do maior valor de $\hat{C_{J}}$, $k = 2$

Logo, vamos tentar substituir $x_7$ artificial por $x_2$ não básica original.

Critérios:

1. $ {A_{\text{artificial}}}_{[7,k]} \neq 0 $

Lembrando que a linha da matriz de restrições correspondente à variável básica $x_7$ neste momento é a linha 2

In [ ]:
md = f'''
$x_2$ { 'pode' if A_artificial[1,2] else 'não pode '} entrar na base.

$ {{A_{{\\text{{artificial}}}}}}_{{[7,2]}} = {A_artificial[1,2]}
$ '''
display(Markdown(md))

Vamos testar agora com o próximo valor de $\hat{C_J}$ que corresponde ao 3º índice, $k = 3$

In [ ]:
md = f'''
$x_3$ { 'pode' if A_artificial[1,3] else 'não pode '} entrar na base.

$ {{A_{{\\text{{artificial}}}}}}_{{[7,3]}} = {A_artificial[1,3]}
$ '''
display(Markdown(md))

Vamos testar agora com o último valor possível de $\hat{C_J}$ que corresponde ao 4º índice, $k = 4$

In [ ]:
md = f'''
$x_4$ { 'pode' if A_artificial[1,4] else 'não pode '} entrar na base.

$ {{A_{{\\text{{artificial}}}}}}_{{[7,4]}} = {A_artificial[1,4]}
$ '''
display(Markdown(md))

Ainda existem mais critérios que devem ser levados em consideração:

2. Se a matriz $A_I$ resultante é não-singular (invertível)

In [ ]:
I = [5, 4, 0, 6]
A_I = A_artificial[:,I]

md = f'''
$A_I$ resultante é {
    'singular.' if np.linalg.det(A_I) == 0 else 'não singular.'
}
'''
display(Markdown(md))

3. Por fim, se os critérios de não negatividade — $ X_I \geq 0 $ — são respeitados, onde:

$$
    X_I = {A_I}^{-1} B
$$

In [ ]:
md = f'''
Os critérios de negatividade{
    ' não ' if np.any(np.linalg.inv(A_I).dot(B) < 0) else ' '
}foram respeitados
'''
display(Markdown(md))
display(np.linalg.inv(A_I).dot(B))

Então, a substituição é de $x_7$ por $x_4$ é válida e vamos utilizar $ I = [5, 4, 0, 6] $ como base inicial factível para o simplex tableau revisto de forma a solucionar o problema original:

In [ ]:
I = [5, 4, 0, 6]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [3, 4, 5, 6]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.5

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [3,  4, 1,  0],
    [1, -1, 0, -1]
])

B = np.array([
    12,
    2
])

C = np.array([1, 2, 0, 0])

I = list(range(2, 4))

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [2, 4]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $ e $x_4 $ (artificial) $ = 0$, o que caracteriza o problema original como factível.

E na base ótima inexistem variáveis artificiais, portanto $ I = [2, 0] $ é base inicial para o problema original

In [ ]:
I = [2, 0]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [2, 0]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)

# Exercicio 4.6

Verificando se as restrições de não negatividade para $ {A_{I}}^{-1} $ são violadas

In [ ]:
A = np.array([
    [2, 4, 1, -1, 1,  0],
    [2, 2, 3, -3, 0, -1]
])

B = np.array([
    6,
    2
])

C = np.array([-5, 2, -2, 1, 0, 0])

I = list(range(4, 6))

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [ ]:
m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [4, 6]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

Verificamos que $ \phi^{*} = 0 $ e $x_4 $ (artificial) $ = 0$, o que caracteriza o problema original como factível.

E na base ótima inexistem variáveis artificiais, portanto $ I = [4, 2] $ é base inicial para o problema original

In [ ]:
I = [4, 2]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)

In [ ]:
solution[2]['y_k']

## Result Comparison through scipy

In [ ]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

## Result Comparison through Simplex Two Phases (without tableaus)

In [ ]:
I = [2, 0]
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)

## Final Solution

In [ ]:
I = [3, 0]
x_2 = symbols('x_2')
x_I = np.array([
    2,
    4
]) - x_2 * np.array([
    -1,
    0
])
x_ = np.zeros(n, dtype=object)
x_[I] = x_I
x_[2] = x_2
display(C @ x_)

# Exercício 4.7

In [11]:
slack_excess_matrix = np.eye(3)
excess_indices = list(range(3))
for excess_index in excess_indices:
    slack_excess_matrix[
        excess_index, excess_index
    ] = -1

A = np.array([
    [1, -2],
    [1,  3],
    [2,  1]
])

A = np.hstack((A, slack_excess_matrix))

B = np.array([
    2,
    4,
    6
])
C = np.array([
    1, 1, 0, 0, 0
])
I = list(range(2,5))

I_sanitized, A_I_sanitized, A_sanitized, B_sanitized, feasible, iters_count, debug_info = _simplex_find_feasible_initial_basis(A, B, C, I)

solution = revised_simplex_tableau(A, B, C, list(I_sanitized))

results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  | \\(A_I^{-1}\\) |   | RHS |
|---|---|---|---|---|
| Z | 0.000 | 0.200 | 0.400 | 3.200 |
| \\(X_{0}\\) | 0.000 | -0.200 | 0.600 | 2.800 |
| \\(X_{1}\\) | 0.000 | 0.400 | -0.200 | 0.400 |
| \\(X_{2}\\) | -1.000 | -1.000 | 1.000 | 0.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
1.000 & -2.000 & -1.000 & 0.000 & 0.000 \\
1.000 & 3.000 & 0.000 & -1.000 & 0.000 \\
2.000 & 1.000 & 0.000 & 0.000 & -1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
2.000\\
4.000\\
6.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
1.000 & 1.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['2.800', '0.400', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 3.200\\)

Optimal Basis: \\([0, 1, 2]\\)

Final non-basic variables set J: \\([3, 4]\\)

\\(\hat{C}_{J}\\): \\([-0.2, -0.4]\\)

Number of Iterations: 1



## Result Comparison through scipy

In [4]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

Result comparison, we found the optimal solution through our tableaus


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 3.1999999999999997
              x: [ 2.800e+00  4.000e-01  0.000e+00  0.000e+00  0.000e+00]
            nit: 1
          lower:  residual: [ 2.800e+00  4.000e-01  0.000e+00  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  4.000e-01  6.000e-01
                              0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [ 4.000e-01  6.000e-01 -0.000e+00]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [10]:
A @ solution[0]['X']

array([2., 4., 6.])

# Exercício 4.28

In [12]:
slack_excess_matrix = np.eye(6)
excess_indices = list(range(3,6))
for excess_index in excess_indices:
    slack_excess_matrix[
        excess_index, excess_index
    ] = -1

A = np.array([
    [0, 1, 2],
    [1, 1, 1],
    [2, 1, 1],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

A = np.hstack((A, slack_excess_matrix))

B = np.array([
    240,
    400,
    380,
    100,
    80,
    60
])

C = np.array([2, 3, 4, 0, 0, 0, 0, 0, 0])

I = list(range(3, 9))

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Restrições de negatividade violadas

Como as restrições de negatividade foram violadas, devemos criar um problema artificial e resolvê-lo para:

1. Verificar a factibilidade do problema original
2. Encontrar uma base inicial factível para o problema original que seja capaz de resolvê-lo

O problema artificial é do tipo:

$$
\begin{aligned}
    & \text{min} \quad \phi = & \textbf{1} \: x_a \\
    & \text{s. t.} & A x + x_a &= b \\
    &              &       x   &\geq 0 \\
    &              &       x_a &\geq 0 \\
\end{aligned}
$$

Vamos resolvê-lo abaixo:

In [13]:

m, n = A.shape

# adicionando variaveis artificiais à matriz de restricoes do problema
# para substituir variaveis de excesso

p = n - m
colunas_variaveis_folga_excesso = range(p, n)
variaveis_artificiais = []
for c in colunas_variaveis_folga_excesso:
    if np.any(A[:,c] < 0):
        # estamos diante de uma variavel de excesso que demanda adicao de
        # variavel artificial
        variavel_artificial = -1 * np.copy(A[:,c])
        variaveis_artificiais.append(variavel_artificial)

# montando array de variaveis artificiais para concatenacao em A
to_stack = np.zeros((m, len(variaveis_artificiais)))
for c, variavel_artificial in enumerate(variaveis_artificiais):
    to_stack[:, c] = np.copy(variavel_artificial)
# montando matriz de restricoes com variaveis artificiais
A_artificial = np.hstack((A, to_stack))

# construindo vetor C artificial
C_artificial = np.zeros(n + len(variaveis_artificiais))
for i in range(n, n + len(variaveis_artificiais)):
    C_artificial[i] = 1

I = [3, 4, 5, 9, 10, 11]

# resolvendo problema artificial com o simplex tableau revisado
solution = revised_simplex_tableau(A_artificial, B, C_artificial, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 1.000 | 240.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 240.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 400.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 380.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{0}\\)
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 1.000 | 240.000 | 1.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 240.000 | 0.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 400.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 0.000 | 380.000 | 2.000 |
| \\(X_{9}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 | 1.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 | 0.000 |


\\(Ĉ_J = \ \\)array([ 1.,  1.,  1., -1., -1., -1.])


\\(J = {[0, 1, 2, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{0}\\)
Variable to leave: \\(X_{9}\\)
#### Row Operations:

1. \\(R_{X_{9}} \leftarrow \frac{R_{X_{9}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{9}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} - 0.000R_{X_{9}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.000R_{X_{9}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 2.000R_{X_{9}}\\)
5. \\(R_{X_{10}} \leftarrow R_{X_{10}} - 0.000R_{X_{9}}\\)
6. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 0.000R_{X_{9}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 140.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 240.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 300.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | 0.000 | 0.000 | 180.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


## Iteration 1

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 140.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 240.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 300.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | 0.000 | 0.000 | 180.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{1}\\)
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 1.000 | 140.000 | 1.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 240.000 | 1.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | 0.000 | 0.000 | 300.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | 0.000 | 0.000 | 180.000 | 1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 | 0.000 |
| \\(X_{10}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 | 1.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 | 0.000 |


\\(Ĉ_J = \ \\)array([-1.,  1.,  1.,  0., -1., -1.])


\\(J = {[9, 1, 2, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{1}\\)
Variable to leave: \\(X_{10}\\)
#### Row Operations:

1. \\(R_{X_{10}} \leftarrow \frac{R_{X_{10}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{10}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} - 1.000R_{X_{10}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.000R_{X_{10}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.000R_{X_{10}}\\)
5. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.000R_{X_{10}}\\)
6. \\(R_{X_{11}} \leftarrow R_{X_{11}} - 0.000R_{X_{10}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 160.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | -1.000 | 0.000 | 220.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | -1.000 | 0.000 | 100.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


## Iteration 2

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 160.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | -1.000 | 0.000 | 220.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | -1.000 | 0.000 | 100.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


Extended Tableau

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS | \\(X_{2}\\)
|---|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 | 1.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | 0.000 | 160.000 | 2.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | -1.000 | 0.000 | 220.000 | 1.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | -1.000 | 0.000 | 100.000 | 1.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 | 0.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 | 0.000 |
| \\(X_{11}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 | 1.000 |


\\(Ĉ_J = \ \\)array([-1., -1.,  1.,  0.,  0., -1.])


\\(J = {[9, 10, 2, 6, 7, 8]}\\)

### Pivot Operations

Variable to enter: \\(X_{2}\\)
Variable to leave: \\(X_{11}\\)
#### Row Operations:

1. \\(R_{X_{11}} \leftarrow \frac{R_{X_{11}}}{1.000}\\)
1. \\(R_{Z} \leftarrow R_{Z} - 1.000R_{X_{11}}\\)
2. \\(R_{X_{3}} \leftarrow R_{X_{3}} - 2.000R_{X_{11}}\\)
3. \\(R_{X_{4}} \leftarrow R_{X_{4}} - 1.000R_{X_{11}}\\)
4. \\(R_{X_{5}} \leftarrow R_{X_{5}} - 1.000R_{X_{11}}\\)
5. \\(R_{X_{0}} \leftarrow R_{X_{0}} - 0.000R_{X_{11}}\\)
6. \\(R_{X_{1}} \leftarrow R_{X_{1}} - 0.000R_{X_{11}}\\)


Computed Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | -2.000 | 40.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | -1.000 | -1.000 | 160.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | -1.000 | -1.000 | 40.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


## Iteration 3

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | -2.000 | 40.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | -1.000 | -1.000 | 160.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | -1.000 | -1.000 | 40.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
0.000 & 1.000 & 2.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
1.000 & 1.000 & 1.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
2.000 & 1.000 & 1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & -0.000 & -0.000 & -0.000 \\
1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 & 1.000 & -0.000 & -0.000 \\
0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & -0.000 & 1.000 & -0.000 \\
0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & -0.000 & -0.000 & 1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
240.000\\
400.000\\
380.000\\
100.000\\
80.000\\
60.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 1.000 & 1.000 & 1.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}\\
X_{9}\\
X_{10}\\
X_{11}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal multiple solutions

Optimal Solution: \\(X^{*} = ['100.000', '80.000', '60.000', '40.000', '160.000', '40.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 0.000\\)

Optimal Basis: \\([3, 4, 5, 0, 1, 2]\\)

Final non-basic variables set J: \\([9, 10, 11, 6, 7, 8]\\)

\\(\hat{C}_{J}\\): \\([-1.0, -1.0, -1.0, 0.0, 0.0, 0.0]\\)

Number of Iterations: 4



Verificamos que $ \phi^{*} = 0 $ e $x_4 $ (artificial) $ = 0$, o que caracteriza o problema original como factível.

E na base ótima inexistem variáveis artificiais, portanto $ I = [2, 0] $ é base inicial para o problema original

In [14]:
I = [3, 4, 5, 0, 1, 2]

# resolvendo problema original com o simplex tableau revisado
solution = revised_simplex_tableau(A, B, C, I)
results_md = markdown_repr_T(solution)

display(
    Markdown(results_md)
)



## Iteration 0

Starting Tableau:

|   |  |   |   | \\(A_I^{-1}\\) |   |   | RHS |
|---|---|---|---|---|---|---|---|
| Z | 0.000 | 0.000 | 0.000 | 2.000 | 3.000 | 4.000 | 680.000 |
| \\(X_{3}\\) | 1.000 | 0.000 | 0.000 | 0.000 | -1.000 | -2.000 | 40.000 |
| \\(X_{4}\\) | 0.000 | 1.000 | 0.000 | -1.000 | -1.000 | -1.000 | 160.000 |
| \\(X_{5}\\) | 0.000 | 0.000 | 1.000 | -2.000 | -1.000 | -1.000 | 40.000 |
| \\(X_{0}\\) | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 0.000 | 100.000 |
| \\(X_{1}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 0.000 | 80.000 |
| \\(X_{2}\\) | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 1.000 | 60.000 |


## Solution for 

\\[
    \begin{aligned}
    & \text{Minimize} & C^{T} \cdot X \\
    & \text{Subject to} & A \cdot X & = B \\
    & & X & \geq 0
    \end{aligned}
    \\]
    where: 
    \\[
    \begin{aligned}
    & & A = \begin{bmatrix}
0.000 & 1.000 & 2.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
1.000 & 1.000 & 1.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 \\
2.000 & 1.000 & 1.000 & 0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 \\
1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 & 0.000 \\
0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 & 0.000 \\
0.000 & 0.000 & 1.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & -1.000 \\
\end{bmatrix} \\
    & & B = \begin{bmatrix}
240.000\\
400.000\\
380.000\\
100.000\\
80.000\\
60.000
\end{bmatrix} \\
    & & C^{T} = \begin{bmatrix}
2.000 & 3.000 & 4.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000 & 0.000
\end{bmatrix} \\
    & & X = \begin{bmatrix}
X_{0}\\
X_{1}\\
X_{2}\\
X_{3}\\
X_{4}\\
X_{5}\\
X_{6}\\
X_{7}\\
X_{8}
\end{bmatrix}
    \end{aligned}
    \\]

Solution Type: Optimal unique solution

Optimal Solution: \\(X^{*} = ['100.000', '80.000', '60.000', '40.000', '160.000', '40.000', '0.000', '0.000', '0.000']\\)

Optimal Value: \\(Z^{*} = 680.000\\)

Optimal Basis: \\([3, 4, 5, 0, 1, 2]\\)

Final non-basic variables set J: \\([6, 7, 8]\\)

\\(\hat{C}_{J}\\): \\([-2.0, -3.0, -4.0]\\)

Number of Iterations: 1



## Result Comparison through scipy

In [15]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

Result comparison, we found the optimal solution through our tableaus


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 680.0
              x: [ 1.000e+02  8.000e+01  6.000e+01  4.000e+01  1.600e+02
                   4.000e+01  0.000e+00  0.000e+00  0.000e+00]
            nit: 0
          lower:  residual: [ 1.000e+02  8.000e+01  6.000e+01  4.000e+01
                              1.600e+02  4.000e+01  0.000e+00  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  2.000e+00  3.000e+00
                              4.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.0

## Result Comparison through Simplex Two Phases (without tableaus)

In [16]:
I = list(range(3,9))
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)


Solution Type: Optimal finite solution found

$ Z^{*} = 680.0 $

$ X^{*} = \begin{bmatrix}
    100.00 \\
80.00 \\
60.00 \\
40.00 \\
160.00 \\
40.00 \\
0.00 \\
0.00 \\
0.00 \\
\end{bmatrix}
$

$ I^{*} = $ [3 4 5 0 1 2]



# Exercício 4.33

In [24]:
slack_excess_matrix = np.eye(10)
excess_indices = list(range(6,10))
for excess_index in excess_indices:
    slack_excess_matrix[
        excess_index, excess_index
    ] = -1

A = np.array([
    np.concatenate((np.zeros(0), [1, 1], np.zeros(10))),
    np.concatenate((np.zeros(2), [1, 1], np.zeros(8))),
    np.concatenate((np.zeros(4), [1, 1], np.zeros(6))),
    np.concatenate((np.zeros(6), [1, 1], np.zeros(4))),
    np.concatenate((np.zeros(8), [1, 1], np.zeros(2))),
    np.concatenate((np.zeros(10), [1, 1], np.zeros(0))),
    np.concatenate((np.zeros(0), [1, 0, 1, 0, 1, 0], np.zeros(6))),
    np.concatenate((np.zeros(0), [0, 1, 0, 1, 0, 1], np.zeros(6))),
    np.concatenate((np.zeros(6), [1, 0, 1, 0, 1, 0], np.zeros(0))),
    np.concatenate((np.zeros(6), [0, 1, 0, 1, 0, 1], np.zeros(0))),
])

A = np.hstack((A, slack_excess_matrix))

B = np.array([
    60,
    80,
    50,
    80,
    50,
    50,
    100,
    90,
    60,
    120
])

C = np.array([4, 5, 2, 4, 5, 3, 4, 5, 2, 4, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

I = list(range(12,22))

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Restrições de negatividade violadas

## Result Comparison through Simplex Two Phases (without tableaus)

In [25]:
I = list(range(12,22))
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)


Solution Type: Optimal finite solution found

$ Z^{*} = 1230.0 $

$ X^{*} = \begin{bmatrix}
    20.00 \\
40.00 \\
80.00 \\
0.00 \\
0.00 \\
50.00 \\
10.00 \\
70.00 \\
50.00 \\
0.00 \\
0.00 \\
50.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
\end{bmatrix}
$

$ I^{*} = $ [0, 1, 5, 7, 8, 11, 2, 12, 6, 15]



## Result Comparison through scipy

In [26]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

Result comparison, we found the optimal solution through our tableaus


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 1230.0
              x: [ 2.000e+01  4.000e+01 ...  0.000e+00  0.000e+00]
            nit: 12
          lower:  residual: [ 2.000e+01  4.000e+01 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  4.000e+00
                              5.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [-0.000e+00 -2.000e+00 -2.000e+00 -0.000e+00
                             -2.000e+00 -2.000e+00  4.000e+00  5.000

# Exercício 4.34

In [5]:
slack_excess_matrix = np.eye(4)
excess_indices = list(range(2,4))
for excess_index in excess_indices:
    slack_excess_matrix[
        excess_index, excess_index
    ] = -1

A = np.array([
    [  1,   1,   1,  1,   1],
    [120, 150, 100, 80, 120],
    [ 10,   0,   0,  0,   0],
    [  0,   4,   0,  0,   0],
])

A = np.hstack((A, slack_excess_matrix))

B = np.array([
    200,
    18000,
    250,
    80
])

C = np.array([-1200, -600, -320, -480, -450, 0, 0, 0, 0])

I = list(range(5, 9))

A_I = A[:,I] # definição da matriz básica
A_I_inv = np.linalg.inv(A_I)

result = f'Restrições de negatividade{
    ' não ' if np.all(A_I_inv @ B >= 0) else ' '
}violadas'

display(
    Markdown(
        result
    )
)

Restrições de negatividade violadas

## Result Comparison through Simplex Two Phases (without tableaus)

In [6]:
_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)


Solution Type: Optimal finite solution found

$ Z^{*} = -162000.0 $

$ X^{*} = \begin{bmatrix}
    125.00 \\
20.00 \\
0.00 \\
0.00 \\
0.00 \\
55.00 \\
0.00 \\
1000.00 \\
0.00 \\
\end{bmatrix}
$

$ I^{*} = $ [5 7 0 1]



## Result Comparison through scipy

In [7]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

Result comparison, we found the optimal solution through our tableaus


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -162000.0
              x: [ 1.250e+02  2.000e+01  0.000e+00  0.000e+00  0.000e+00
                   5.500e+01  0.000e+00  1.000e+03  0.000e+00]
            nit: 2
          lower:  residual: [ 1.250e+02  2.000e+01  0.000e+00  0.000e+00
                              0.000e+00  5.500e+01  0.000e+00  1.000e+03
                              0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  6.800e+02  3.200e+02
                              7.500e+02  0.000e+00  1.000e+01  0.000e+00
                              2.250e+02]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00 

# Comparing the simplex2 method as intended by Dr. Sergio Ricardo de Souza

In [8]:
from simplex_two_phases import simplex2

z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex2(A, B, C, debug=True)

X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)


Solution Type: Optimal finite solution found

$ Z^{*} = -162000.0 $

$ X^{*} = \begin{bmatrix}
    125.00 \\
20.00 \\
0.00 \\
0.00 \\
0.00 \\
55.00 \\
0.00 \\
1000.00 \\
0.00 \\
\end{bmatrix}
$

$ I^{*} = $ [5 7 0 1]



# Exemplo com restrições redundantes

In [8]:
A = np.array([
    [1,  1, 1, -1,  0,  0, 0],
    [-1, 1, 2,  0, -1,  0, 0],
    [ 0, 2, 3,  0,  0, -1, 0],
    [ 0, 0, 1,  0,  0,  0, 1]
])

B = np.array([
    6,
    4,
    10,
    2
])

C = np.array([
    -1, 2, -3, 0, 0, 0, 0
])

I = [3, 4, 5, 6]

_, n = A.shape
X = np.zeros(n)
z_star, x_star, I_star, A_I_star, A, iterations_count, solution_type, debug_info  = simplex_two_phases(A, B, C, I, debug=True)
X[I_star] = x_star

result_md = f'''
Solution Type: {solution_types[solution_type]}

$ Z^{{*}} = {z_star} $

$ X^{{*}} = \\begin{{bmatrix}}
    { '\n'.join([f'{x:.2f} \\\\' for x in X]) }
\\end{{bmatrix}}
$

$ I^{{*}} = $ {I_star}

'''
display(
    Markdown(
        result_md
    )
)

[7 8 9 6]
[0 1 3 6]



Solution Type: Optimal finite solution found

$ Z^{*} = -4.0 $

$ X^{*} = \begin{bmatrix}
    2.00 \\
2.00 \\
2.00 \\
0.00 \\
0.00 \\
0.00 \\
0.00 \\
\end{bmatrix}
$

$ I^{*} = $ [0 1 3 2]



In [9]:
result = linprog(C, A_eq = A, b_eq = B, method='highs')
print('Result comparison, we found the optimal solution through our tableaus')
display(result)

Result comparison, we found the optimal solution through our tableaus


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -4.0
              x: [ 2.000e+00  2.000e+00  2.000e+00 -0.000e+00  0.000e+00
                   0.000e+00  0.000e+00]
            nit: 2
          lower:  residual: [ 2.000e+00  2.000e+00  2.000e+00 -0.000e+00
                              0.000e+00  0.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              1.000e+00  5.000e-01  6.500e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-0.000e+00  1.000e+00  5.000e-01 -6.500e+00]
        ineqlin:  residual: []
